In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

data_dir = os.path.join(os.getcwd(), "dataset")
train_data = datasets.ImageFolder(data_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)

class_names = train_data.classes
print(class_names)

images, labels = next(iter(train_loader))
print("이미지 배치 shape:", images.shape)
print("레이블:", labels)

['cat', 'dog', 'fox', 'rabbit']
이미지 배치 shape: torch.Size([2, 3, 64, 64])
레이블: tensor([2, 0])


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class MultipleClassificationModel(nn.Module):
    def __init__(self, num_classes):
        """
        모델의 초기화 함수, 모델에 사용할 각 계층을 정의
        num_classes : 분류할 클래스의 수
        """
        super().__init__()
        
        #  첫 번째 Convolution Block
        # Conv2d :  이미지에서 특성을 추출하는 층
        # ( 입력 채널 : 3, 출력 채널 : 32, 커널 크기 : 3 x 3 )  / 커널(= 필터, 마스크) : 이미지로부터 원하는 특징만 추출하는 것
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1) # RGB 이미지 ( 채널 3 ) 을 입력받고, 32개의 특성 맵을 추출
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 32 개 특성 맵을 입력받고, 64개로 확장
        self.pool = nn.MaxPool2d(2, 2) # Max Pooling : 특성 맵의 크기를 절반으로 줄임 ( => 계산량 감소 )
        
        # 두 번째 Convolution Block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 64개의 특성 맵을 받아 128개로 확장
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1) # 128개의 특성 맵을 받아 256개로 확장
        
        # Fully Connected Layers ( FC ) : 마지막에 나온 특성 맵을 1차원 벡터로 펼쳐서 분류 작업을 시행
        self.fc1 = nn.Linear(256 * 16 * 16, 1024) # 마지막 특성 맵을 1024개의 뉴런으로 변환
        self.fc2 = nn.Linear(1024, num_classes) # 1024개의 뉴런을 받아 최종 클래스 개수에 맞는 출력값을 생성'
    
    def forward(self, x):
        """
        입력 데이터 x를 모델을 통해 순차적으로 전달하며 예측을 생성
        """
        
        #  첫 번째 Convolution Block
        x = F.relu(self.conv1(x)) # 첫 번째 Conv2d + ReLU 활성화 함수
        x = self.pool(F.relu(self.conv2(x))) # 두 번째 Conv2d + ReLU + Max pooling
        
        x = F.relu(self.conv3(x)) # 세 번째 Conv2d + ReLU 활성화 함수
        x = self.pool(F.relu(self.conv4(x))) # 네 번째 Conv2d + ReLU + Max pooling
        
        # 전역 평균 풀링 (  Global Average Pooling ) : 특성 맵을 1차원 벡터로 평탄화
        x = x.view(-1, 256 * 16 * 16) # 특성 맵을 1차원 벡터로 펼침
        
        # Fully Connected Layers ( FC ) : 최종 분류 결과를 출력
        x = F.relu(self.fc1(x)) # 첫 번째 FC + ReLU 활성화 함수
        x = self.fc2(x) # 두 번째 FC : 최송 클래스에 대한 예측 결과
        return x

In [4]:
import torch.optim as optim
from torch.utils.data import DataLoader
import torch

num_classes = len(class_names)
model = MultipleClassificationModel(num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images,labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss : {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%')

Epoch [1/20], Loss : 1.5805, Accuracy: 25.00%
Epoch [2/20], Loss : 1.4133, Accuracy: 20.00%
Epoch [3/20], Loss : 1.3869, Accuracy: 25.00%
Epoch [4/20], Loss : 1.3869, Accuracy: 25.00%
Epoch [5/20], Loss : 1.3865, Accuracy: 25.00%
Epoch [6/20], Loss : 1.3871, Accuracy: 30.00%
Epoch [7/20], Loss : 1.3862, Accuracy: 25.00%
Epoch [8/20], Loss : 1.3902, Accuracy: 27.50%
Epoch [9/20], Loss : 1.3861, Accuracy: 27.50%
Epoch [10/20], Loss : 1.4734, Accuracy: 35.00%
Epoch [11/20], Loss : 1.3676, Accuracy: 20.00%
Epoch [12/20], Loss : 1.2968, Accuracy: 27.50%
Epoch [13/20], Loss : 1.3810, Accuracy: 55.00%
Epoch [14/20], Loss : 1.0087, Accuracy: 67.50%
Epoch [15/20], Loss : 1.0567, Accuracy: 70.00%
Epoch [16/20], Loss : 0.5389, Accuracy: 77.50%
Epoch [17/20], Loss : 0.4590, Accuracy: 82.50%
Epoch [18/20], Loss : 0.8881, Accuracy: 85.00%
Epoch [19/20], Loss : 0.6935, Accuracy: 72.50%
Epoch [20/20], Loss : 0.2431, Accuracy: 95.00%


In [6]:
model_path = os.path.join(os.getcwd(), "model/animal_face.pth")

torch.save(model.state_dict(), model_path)